In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from keras.layers import Dense, Dropout
from keras.models import Model, Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import time

In [2]:
df = pd.read_csv("luka_dataset.csv",index_col=0)
df.head()

G  FG  FGA    FG%  3P  3PA    3P%  FT  FTA    FT%  ...  BLK  TOV  PF  PTS  \
0  1   5   16  0.313   0    5  0.000   0    2  0.000  ...    0    4   3   10   
1  2   8   16  0.500   4    9  0.444   6    9  0.667  ...    1    6   3   26   
2  3   7   14  0.500   3    8  0.375   2    2  1.000  ...    0    3   3   19   
3  4   7   18  0.389   2    9  0.222   5    8  0.625  ...    1    3   2   21   
4  5   7   14  0.500   4    6  0.667   4    4  1.000  ...    0    3   1   22   

   GmSc  "+/-"  Win  Winby    Minutes      age_y  
0   0.8    -18    0    -21  31.783333  19.632877  
1  16.6      6    1      4  36.233333  19.641096  
2  13.9      5    1      6  35.800000  19.646575  
3  12.0    -12    0     -7  34.183333  19.652055  
4  16.3      0    0     -9  35.150000  19.657534  

[5 rows x 25 columns]

In [3]:
df = df.astype('float32')

In [4]:
y = df['Win']
X = df.drop(columns=['Win','Winby'])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=42)

In [6]:
def my_DL(epochs=6,batchsize=512):
    model = Sequential()
    model.add(Dense(32,activation='relu'))
    model.add(Dense(16,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
    return model

In [7]:
def train_hyper_tune(X,y):
    # create the pre-processing component
    my_scaler = StandardScaler()
    my_imputer = SimpleImputer(strategy="median")
    
    # define classifiers
    ## Classifier 1: Logistic Regression
    clf_LR = LogisticRegression(random_state=0,penalty='elasticnet',solver='saga')
    ## Classifier 2: Random Forest Classifier
    clf_RF = RandomForestClassifier(random_state=0)
    ## Classifier 3: Deep Learning Binary Classifier
    clf_DL = KerasClassifier(build_fn=my_DL)
    
    # define pipeline for three classifiers
    ## clf_LR
    pipe1 = Pipeline([('imputer', my_imputer), ('scaler', my_scaler), ('lr_model',clf_LR)])
    ## clf_RF
    pipe2 = Pipeline([('imputer', my_imputer), ('scaler', my_scaler), ('rf_model',clf_RF)])
    ## clf_DL
    pipe3 = Pipeline([('imputer', my_imputer), ('scaler', my_scaler), ('dl_model',clf_DL)])
    
    # create hyperparameter space of the three models
    ## clf_LR
    param_grid1 = {
        'lr_model__C' : [1e-1,1,10],
        'lr_model__l1_ratio' : [0,0.5,1]
    }
    ## clf_RF
    param_grid2 = {
        'rf_model__n_estimators' : [50,100],
        'rf_model__max_features' : [0.8,"auto"],
        'rf_model__max_depth' : [4,5]
    }
    ## clf_DL
    param_grid3 = {
        'dl_model__epochs' : [6,12,18,24],
        'dl_model__batchsize' : [256,512]
    }
    
    # set GridSearch via 5-fold cross-validation
    ## clf_LR
    grid1 = GridSearchCV(pipe1, cv=5, param_grid=param_grid1)
    ## clf_RF
    grid2 = GridSearchCV(pipe2, cv=5, param_grid=param_grid2)
    ## clf_DL
    grid3 = GridSearchCV(pipe3, cv=5, param_grid=param_grid3)
    
    # run the hyperparameter tunning process
    start_LR = time.perf_counter()
    grid1.fit(X,y)
    finish_LR = time.perf_counter()
    start_RF = time.perf_counter()
    grid2.fit(X,y)
    finish_RF = time.perf_counter()
    start_DL = time.perf_counter()
    grid3.fit(X,y)
    finish_DL = time.perf_counter()
    print("Training Time For LR: ",finish_LR-start_LR, "seconds")
    print("Training Time For RF: ",finish_RF-start_RF, "seconds")
    print("Training Time For DL: ",finish_DL-start_DL, "seconds")
    # return results of the tunning process
    return grid1,grid2,grid3,pipe1,pipe2,pipe3

In [8]:
my_grid1,my_grid2,my_grid3,my_pipe1,my_pipe2,my_pipe3 = train_hyper_tune(X_train, y_train)

/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sa

Epoch 1/6
3/3 [==============================] - 0s 3ms/step - loss: 0.6870 - accuracy: 0.5568
Epoch 2/6
3/3 [==============================] - 0s 4ms/step - loss: 0.6363 - accuracy: 0.6364
Epoch 3/6
3/3 [==============================] - 0s 3ms/step - loss: 0.6130 - accuracy: 0.6818
Epoch 4/6
3/3 [==============================] - 0s 5ms/step - loss: 0.5978 - accuracy: 0.7386
Epoch 5/6
3/3 [==============================] - 0s 3ms/step - loss: 0.5817 - accuracy: 0.7273
Epoch 6/6
1/1 [==============================] - 0s 1ms/step - loss: 0.5654 - accuracy: 0.6957
Epoch 1/6
3/3 [==============================] - 0s 1ms/step - loss: 0.7375 - accuracy: 0.5056
Epoch 2/6
3/3 [==============================] - 0s 2ms/step - loss: 0.6959 - accuracy: 0.5281
Epoch 3/6
3/3 [==============================] - 0s 2ms/step - loss: 0.6741 - accuracy: 0.5618
Epoch 4/6
3/3 [==============================] - 0s 8ms/step - loss: 0.6582 - accuracy: 0.5730
Epoch 5/6
3/3 [==============================] - 0

3/3 [==============================] - 0s 2ms/step - loss: 0.7353 - accuracy: 0.4494
Epoch 3/12
3/3 [==============================] - 0s 3ms/step - loss: 0.7099 - accuracy: 0.4831
Epoch 4/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6904 - accuracy: 0.5393
Epoch 5/12
3/3 [==============================] - 0s 3ms/step - loss: 0.6734 - accuracy: 0.5955
Epoch 6/12
3/3 [==============================] - 0s 4ms/step - loss: 0.6590 - accuracy: 0.6404
Epoch 7/12
3/3 [==============================] - 0s 1ms/step - loss: 0.6462 - accuracy: 0.6629
Epoch 8/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6329 - accuracy: 0.7079
Epoch 9/12
3/3 [==============================] - 0s 3ms/step - loss: 0.6202 - accuracy: 0.7416
Epoch 10/12
3/3 [==============================] - 0s 6ms/step - loss: 0.6092 - accuracy: 0.7640
Epoch 11/12
3/3 [==============================] - 0s 3ms/step - loss: 0.5989 - accuracy: 0.7865
Epoch 12/12
1/1 [==============================] 

1/1 [==============================] - 0s 932us/step - loss: 0.6098 - accuracy: 0.6957
Epoch 1/18
3/3 [==============================] - 0s 1ms/step - loss: 0.7125 - accuracy: 0.5056
Epoch 2/18
3/3 [==============================] - 0s 1ms/step - loss: 0.6672 - accuracy: 0.5843
Epoch 3/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6444 - accuracy: 0.6180
Epoch 4/18
3/3 [==============================] - 0s 3ms/step - loss: 0.6213 - accuracy: 0.6404
Epoch 5/18
3/3 [==============================] - 0s 3ms/step - loss: 0.6030 - accuracy: 0.6517
Epoch 6/18
3/3 [==============================] - 0s 3ms/step - loss: 0.5854 - accuracy: 0.6966
Epoch 7/18
3/3 [==============================] - 0s 2ms/step - loss: 0.5691 - accuracy: 0.6966
Epoch 8/18
3/3 [==============================] - 0s 1ms/step - loss: 0.5534 - accuracy: 0.7528
Epoch 9/18
3/3 [==============================] - 0s 2ms/step - loss: 0.5395 - accuracy: 0.7978
Epoch 10/18
3/3 [==============================] 

3/3 [==============================] - 0s 4ms/step - loss: 0.7508 - accuracy: 0.4944
Epoch 5/18
3/3 [==============================] - 0s 4ms/step - loss: 0.7242 - accuracy: 0.4944
Epoch 6/18
3/3 [==============================] - 0s 4ms/step - loss: 0.7025 - accuracy: 0.5056
Epoch 7/18
3/3 [==============================] - 0s 5ms/step - loss: 0.6824 - accuracy: 0.5056
Epoch 8/18
3/3 [==============================] - 0s 4ms/step - loss: 0.6663 - accuracy: 0.5281
Epoch 9/18
3/3 [==============================] - 0s 4ms/step - loss: 0.6501 - accuracy: 0.5506
Epoch 10/18
3/3 [==============================] - 0s 4ms/step - loss: 0.6370 - accuracy: 0.5506
Epoch 11/18
3/3 [==============================] - 0s 4ms/step - loss: 0.6236 - accuracy: 0.5730
Epoch 12/18
3/3 [==============================] - 0s 3ms/step - loss: 0.6129 - accuracy: 0.6067
Epoch 13/18
3/3 [==============================] - 0s 5ms/step - loss: 0.6026 - accuracy: 0.6067
Epoch 14/18
3/3 [==============================

...

3/3 [==============================] - 0s 1ms/step - loss: 0.7528 - accuracy: 0.4270
Epoch 3/12
3/3 [==============================] - 0s 2ms/step - loss: 0.7324 - accuracy: 0.4494
Epoch 4/12
3/3 [==============================] - 0s 2ms/step - loss: 0.7154 - accuracy: 0.5056
Epoch 5/12
3/3 [==============================] - 0s 3ms/step - loss: 0.7049 - accuracy: 0.5056
Epoch 6/12
3/3 [==============================] - 0s 3ms/step - loss: 0.6910 - accuracy: 0.5281
Epoch 7/12
3/3 [==============================] - 0s 6ms/step - loss: 0.6795 - accuracy: 0.5393
Epoch 8/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6695 - accuracy: 0.5393
Epoch 9/12
3/3 [==============================] - 0s 3ms/step - loss: 0.6602 - accuracy: 0.5618
Epoch 10/12
3/3 [==============================] - 0s 4ms/step - loss: 0.6496 - accuracy: 0.5955
Epoch 11/12
3/3 [==============================] - 0s 2ms/step - loss: 0.6399 - accuracy: 0.6404
Epoch 12/12
1/1 [==============================] 

Epoch 13/18
3/3 [==============================] - 0s 2ms/step - loss: 0.4803 - accuracy: 0.8090
Epoch 14/18
3/3 [==============================] - 0s 1ms/step - loss: 0.4687 - accuracy: 0.8202
Epoch 15/18
3/3 [==============================] - 0s 2ms/step - loss: 0.4610 - accuracy: 0.8202
Epoch 16/18
3/3 [==============================] - 0s 4ms/step - loss: 0.4524 - accuracy: 0.8315
Epoch 17/18
3/3 [==============================] - 0s 4ms/step - loss: 0.4440 - accuracy: 0.8315
Epoch 18/18
1/1 [==============================] - 0s 3ms/step - loss: 0.6528 - accuracy: 0.6364
Epoch 1/18
3/3 [==============================] - 0s 1ms/step - loss: 0.7910 - accuracy: 0.4382
Epoch 2/18
3/3 [==============================] - 0s 2ms/step - loss: 0.7275 - accuracy: 0.5056
Epoch 3/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6944 - accuracy: 0.5618
Epoch 4/18
3/3 [==============================] - 0s 2ms/step - loss: 0.6679 - accuracy: 0.6067
Epoch 5/18
3/3 [==================

3/3 [==============================] - 0s 1ms/step - loss: 0.5048 - accuracy: 0.8636
Epoch 18/24
3/3 [==============================] - 0s 2ms/step - loss: 0.4949 - accuracy: 0.8750
Epoch 19/24
3/3 [==============================] - 0s 3ms/step - loss: 0.4852 - accuracy: 0.8636
Epoch 20/24
3/3 [==============================] - 0s 2ms/step - loss: 0.4752 - accuracy: 0.9091
Epoch 21/24
3/3 [==============================] - 0s 2ms/step - loss: 0.4658 - accuracy: 0.9091
Epoch 22/24
3/3 [==============================] - 0s 1ms/step - loss: 0.4560 - accuracy: 0.9091
Epoch 23/24
3/3 [==============================] - 0s 1ms/step - loss: 0.4469 - accuracy: 0.9091
Epoch 24/24
1/1 [==============================] - 0s 831us/step - loss: 0.5150 - accuracy: 0.8261
Epoch 1/24
3/3 [==============================] - 0s 1ms/step - loss: 0.6642 - accuracy: 0.6067
Epoch 2/24
3/3 [==============================] - 0s 2ms/step - loss: 0.6154 - accuracy: 0.6854
Epoch 3/24
3/3 [==========================

3/3 [==============================] - 0s 2ms/step - loss: 0.6212 - accuracy: 0.6292
Epoch 4/24
3/3 [==============================] - 0s 2ms/step - loss: 0.6021 - accuracy: 0.6966
Epoch 5/24
3/3 [==============================] - 0s 2ms/step - loss: 0.5867 - accuracy: 0.6966
Epoch 6/24
3/3 [==============================] - 0s 3ms/step - loss: 0.5724 - accuracy: 0.7191
Epoch 7/24
3/3 [==============================] - 0s 3ms/step - loss: 0.5589 - accuracy: 0.7416
Epoch 8/24
3/3 [==============================] - 0s 3ms/step - loss: 0.5488 - accuracy: 0.7528
Epoch 9/24
3/3 [==============================] - 0s 4ms/step - loss: 0.5350 - accuracy: 0.7753
Epoch 10/24
3/3 [==============================] - 0s 3ms/step - loss: 0.5241 - accuracy: 0.7865
Epoch 11/24
3/3 [==============================] - 0s 3ms/step - loss: 0.5127 - accuracy: 0.7865
Epoch 12/24
3/3 [==============================] - 0s 3ms/step - loss: 0.5021 - accuracy: 0.7978
Epoch 13/24
3/3 [==============================]

4/4 [==============================] - 0s 4ms/step - loss: 0.4328 - accuracy: 0.8559
Epoch 23/24
4/4 [==============================] - 0s 4ms/step - loss: 0.4241 - accuracy: 0.8559
Epoch 24/24
4/4 [==============================] - 0s 3ms/step - loss: 0.4142 - accuracy: 0.8649
Training Time For LR:  0.728107793999996 seconds
Training Time For RF:  4.653514879999996 seconds
Training Time For DL:  41.490904404 seconds


In [9]:
def train_on_entire(X,y,pipe,grid_res):
    # fit pipeline
    pipe.set_params(**grid_res.best_params_).fit(X, y)
    # return the newly trained pipeline
    return pipe

In [10]:
start = time.perf_counter()
print("Accuracy for LR: ",train_on_entire(X_train,y_train,my_pipe1,my_grid1).score(X_test,y_test))
finish = time.perf_counter()
print("Execution Time for LR: ",finish-start, "seconds")

Accuracy for LR:  0.6071428571428571
Execution Time for LR:  0.028648896000021296 seconds


In [11]:
start = time.perf_counter()
print("Accuracy for RF: ",train_on_entire(X_train,y_train,my_pipe2,my_grid2).score(X_test,y_test))
finish = time.perf_counter()
print("Execution Time for RF: ",finish-start, "seconds")

Accuracy for RF:  0.6785714285714286
Execution Time for RF:  0.1649771019999946 seconds


In [12]:
start = time.perf_counter()
print("Accuracy for DL: ",train_on_entire(X_train,y_train,my_pipe3,my_grid3).score(X_test,y_test))
finish = time.perf_counter()
print("Execution Time for DL: ",finish-start, "seconds")

Epoch 1/24
4/4 [==============================] - 0s 2ms/step - loss: 0.7759 - accuracy: 0.4234
Epoch 2/24
4/4 [==============================] - 0s 3ms/step - loss: 0.7274 - accuracy: 0.4685
Epoch 3/24
4/4 [==============================] - 0s 5ms/step - loss: 0.6994 - accuracy: 0.4775
Epoch 4/24
4/4 [==============================] - 0s 4ms/step - loss: 0.6793 - accuracy: 0.5676
Epoch 5/24
4/4 [==============================] - 0s 4ms/step - loss: 0.6622 - accuracy: 0.5856
Epoch 6/24
4/4 [==============================] - 0s 8ms/step - loss: 0.6473 - accuracy: 0.6306
Epoch 7/24
4/4 [==============================] - 0s 8ms/step - loss: 0.6329 - accuracy: 0.6486
Epoch 8/24
4/4 [==============================] - 0s 4ms/step - loss: 0.6207 - accuracy: 0.6577
Epoch 9/24
4/4 [==============================] - 0s 3ms/step - loss: 0.6071 - accuracy: 0.7027
Epoch 10/24
4/4 [==============================] - 0s 4ms/step - loss: 0.5957 - accuracy: 0.6937
Epoch 11/24
4/4 [======================